In [1]:
import pandas as pd
import numpy as np

from IPython.display import HTML
from numpy import linalg as lp

from IPython.core.display import Javascript, display

pd.set_option('precision', 1)

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Нажмите на кнопку чтобы увидеть код"></form>''')

## Решение задачи линейного программирования симплекс-методом

### Теоретические сведения

В предыдущем параграфе для решения ОЗЛП был применен графический метод. В общем случае ОЗЛП решается с помощью симплекс-метода, к изложению которого мы сейчас и переходим.
    
&#8195;&#8195; Рассмотрим ОЗЛП с системой ограничений в следующей форме:
   
   \begin{equation*}
       \begin{cases}
           {a_{11}x_1+a_{12}x_2+...+a_{1n}x_n}\leq b_1,\\
           {a_{21}x_1+a_{22}x_2+...+a_{2n}x_n}\leq b_2,\\
           ...............................\\
           {a_{m1}x_1+a_{m2}x_2+...+a_{mn}x_n}\leq b_m,
       \end{cases}
   \end{equation*}
   
&#8195;&#8195; Если ввести в систему ограничений дополнительные переменные:
   
&#8195;&#8195; $x_{n+1}$, $x_{n+2}$, ..., $x_{n+m}$
    
&#8195;&#8195; по формулам:
    
   \begin{equation*}
       \begin{cases}
           x_{n+1}=b_1-{a_{11}x_1+a_{12}x_2+...+a_{1n}x_n}\\
           x_{n+2}=b_2-{a_{21}x_1+a_{22}x_2+...+a_{2n}x_n}\\
            ...................................\\
           x_{n+m}=b_m-{a_{m1}x_1+a_{m2}x_2+...+a_{mn}x_n}\\
       \end{cases}
   \end{equation*}
    
&#8195;&#8195;то система ограничений преобразуется в систему уравнений:
    
   \begin{equation*}
       \begin{cases}
           {a_{11}x_1+a_{12}x_2+...+a_{1n}x_n}=b_1\\
           {a_{21}x_1+a_{22}x_2+...+a_{2n}x_n}=b_2\\
            ................................\\
           {a_{m1}x_1+a_{m2}x_2+...+a_{mn}x_n}=b_m
       \end{cases} 
   \end{equation*}
   
&#8195;&#8195; имеющую специальный вид. В этой системе каждая из переменных $x_{n+1}$, $x_{n+2}$, ..., $x_{n+m}$ исключена из всех уравнений, за исключением одного уравнения, в котором коэффициент при ней равен 1.

### Понятие опорного плана

- Базисным решением ОЗЛП называется такое решение системы уравнений, в котором все свободные переменные равны 0.
- Базисное решение ОЗЛП называется опорным решением (опорным планом), если в нем все базисные переменные неотрицательны.
- В теории симплекс-метода доказывается, что, если максимум целевой функции при данной в ОЗЛП системе ограничений существует, то он достигается на опорном решении.
- Опорное решение, на котором целевая функция достигает максимума, является оптимальным планом.

&#8195;&#8195; Замечание. Не следует думать, что ОЗЛП всегда имеет решение. Система ограничений, например, может быть противоречивой и задавать пустое множество решений. Система ограничений может также задавать неограниченное множество, на котором функция прибыли является неограниченной.

### Расчетный алгоритм симплекс-метода

&#8195;&#8195; Алгоритм симплекс-метода состоит из нескольких этапов: сначала происходит построение одного из опорных решений, а затем - «улучшение» этого решения, т.е. переход к другим опорным решениям, на которых значениецелевой функции не уменьшается. Для проведения расчетов используются симплекс-таблицы, составленные по коэффициентам системы, а процесс поиска и улучшения опорного решения заключается в пересчете элементов этих таблиц методом полных жордановых исключений.

In [3]:
print("Введите количество ограничений")
n=int(input())

print("Введите количество переменных")
m=int(input())

constr = np.zeros((n,m+1), dtype=int)
c = []
count = 0

print('Введите коеффициенты целевой функции через пробел:', sep='\n\n')
c = np.array(input().split()).astype(int)

for i in range(n):
    count+=1
    print("Введите коеффициенты " + str(count) + " ограничения через пробел:")
    constr[i]=input().split()

b = constr[:, len(constr[0]) - 1]
constr = np.delete(constr, len(constr[0]) - 1, axis = 1)

Введите количество ограничений
3
Введите количество переменных
2
Введите коеффициенты целевой функции через пробел:
2 5
Введите коеффициенты 1 ограничения через пробел:
1 1 6
Введите коеффициенты 2 ограничения через пробел:
0 1 3
Введите коеффициенты 3 ограничения через пробел:
1 2 9


In [4]:
class LinearModel:
    
    def __init__(self, A = np.empty([0,0]), b = np.empty([0,0]), c = np.empty([0,0]), minmax = "MAX"):
        self.A = A
        self.b = b
        self.c = c
        self.x = [float(0)] * len(c)
        self.minmax = minmax
        self.printIter = True
        self.optimalValue = None
        self.transform = False
        
    def addA(self, A):
        self.A = A
        
    def addB(self, b):
        self.b = b
        
    def addC(self, c):
        self.c = c
        self.transform = False
    
    def setObj(self, minmax):
        if(minmax == "MIN" or minmax == "MAX"):
            self.minmax = minmax
        else:
            print("Некорректное значение")
        self.transform = False
            
    def setPrintIter(self, printIter):
        self.printIter = printIter
            
    def printSoln(self):
        print("Коеффициенты: " + str(self.x))
        print("")
        print("Оптимальное значение функции: " + str(self.optimalValue))
        
    def printTableau(self, tableau):
        
        tableau1 = tableau
        A = ['F']
        A1 = [''+str(i+1) for i in range(len(tableau1) - 1)]
        A1_NEW = A + A1

        tableau1 = np.transpose(tableau1, axes = None)
        B0 = {'New_X': tableau1[0]}
        B = {'R_S': tableau1[1]}

        B0.update(B)

        B1 = {'x_'+str(j-1): tableau1[j] for j in range(2,len(tableau1) - 3)}

        B2 = {'s_'+str(j-3): tableau1[j] for j in range(4,len(tableau1))}

        B1.update(B2)
        z = dict(list(B0.items()) + list(B1.items()))
        df = pd.DataFrame(index = A1_NEW, data=z)
        
        def hover(hover_color="#ffff99"):
    
            return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

        styles = [
            hover(),
            dict(selector="th", props=[("font-size", "120%"),
                                       ("text-align", "center")]),
            dict(selector="caption", props=[("caption-side", "bottom")]),
            dict(selector="tr", props=[("font-size", "150%"),
                                       ("text-align", "center")])
        ]
        html = (df.style.set_table_styles(styles))
        display(html)
            
    def getTableau(self):
        # Создание стартовой симплекс-таблицы
        
        if(self.minmax == "MIN" and self.transform == False):
            self.c[0:len(c)] = -1 * self.c[0:len(c)]
            self.transform = True
        
        t1 = np.array([None, 0])
        numVar = len(self.c)
        numSlack = len(self.A)
        
        t1 = np.hstack(([None], [0], self.c, [0] * numSlack))
        
        basis = np.array([0] * numSlack)
        
        for i in range(0, len(basis)):
            basis[i] = numVar + i
        
        A = self.A
        
        if(not ((numSlack + numVar) == len(self.A[0]))):
            B = np.identity(numSlack)
            A = np.hstack((self.A, B))
            
        t2 = np.hstack((np.transpose([basis]), np.transpose([self.b]), A))
        
        tableau = np.vstack((t1, t2))
        
        tableau = np.array(tableau, dtype ='float')
        
        return tableau
            
    def optimize(self):
        
        if(self.minmax == "MIN" and self.transform == False):
            for i in range(len(self.c)):
                self.c[i] = -1 * self.c[i]
                transform = True
        
        tableau = self.getTableau()
         
        if(self.printIter == True):
            print("Начальная симплекс-таблица:")
            self.printTableau(tableau)
        
        optimal = False

        iter = 1

        while(True):
            
            if(self.printIter == True):
                print("_________________________")
                print("")
                print("Итерация :", iter)
                self.printTableau(tableau)
                
            if(self.minmax == "MAX"):
                for profit in tableau[0, 2:]:
                    if profit > 0:
                        optimal = False
                        break
                    optimal = True
            else:
                for cost in tableau[0, 2:]:
                    if cost < 0:
                        optimal = False
                        break
                    optimal = True

            if optimal == True: 
                 break
            
            
            if (self.minmax == "MAX"):
                n = tableau[0, 2:].tolist().index(np.amax(tableau[0, 2:])) + 2
            else:
                n = tableau[0, 2:].tolist().index(np.amin(tableau[0, 2:])) + 2

            minimum = 99999
            r = -1

            for i in range(1, len(tableau)): 
                if(tableau[i, n] > 0):
                    val = tableau[i, 1]/tableau[i, n]
                    if val<minimum: 
                        minimum = val 
                        r = i
                            
            pivot = tableau[r, n] 
            
            print("Ведущий столбец:", n)
            print("Ведущая строка:", r)
            print("Разрешающий элемент: ", pivot)

        
            tableau[r, 1:] = tableau[r, 1:] / pivot 
            
            

            # pivot other rows
            for i in range(0, len(tableau)): 
                if i != r:
                    mult = tableau[i, n] / tableau[r, n]
                    tableau[i, 1:] = tableau[i, 1:] - mult * tableau[r, 1:] 


             
            tableau[r, 0] = n - 2
            
            iter += 1
            
        
        if(self.printIter == True):
            print("_________________________")
            print("")
            print("Окончательный вариант получен за ", iter, " итераций(ии)")
            self.printTableau(tableau)
        else:
            print("Посчитано")
            
        self.x = np.array([0] * len(c), dtype = float)
        
        for key in range(1, (len(tableau))):
            if(tableau[key, 0] < len(c)):
                self.x[int(tableau[key, 0])] = tableau[key, 1]
        
        self.optimalValue = -1 * tableau[0,1]
        
        return tableau

### Разберем практическое решение

#### Шаг №1 Составление симплекс-таблицы и опорного плана


&#8195;&#8195; Решим прямую задачу линейного программирования симплексным методом, с использованием симплексной таблицы.
Определим максимальное значение целевой функции при заданных ранее ограничениях.

&#8195;&#8195; Для построения первого опорного плана систему неравенств приведем к системе уравнений путем введения дополнительных переменных (переход к канонической форме).

&#8195;&#8195; В 1-м неравенстве вводим базисную переменную x3. В 2-м неравенстве вводим базисную переменную x4. В 3-м неравенстве вводим базисную переменную x5. Базисные переменные это переменные, которые входят только в одно уравнение системы ограничений и притом с единичным коэффициентом. Базисное решение называется допустимым, если оно неотрицательно.
Решим систему уравнений относительно базисных переменных: x3, x4, x5
Полагая, что свободные переменные равны 0, получим первый опорный план. Если этот план нам подходит, то можем переходить к основному алгоритму симплекс-метода

In [5]:
model1 = LinearModel()

model1.addA(constr)
model1.addB(b)
model1.addC(c)

temp = model1.getTableau()
model1.printTableau(temp)

,New_X,R_S,x_1,x_2,s_1,s_2,s_3
F,nan,0,2,5,0,0,0
1,2,6,1,1,1,0,0
2,3,3,0,1,0,1,0
3,4,9,1,2,0,0,1


#### Шаг №2 Посик оптимального решения

&#8195;&#8195; Рассмотрим элементы целевой (фиктивной целевой) строки, стоящие в столбцах $x_1$, ..., $x_{n+m}$. Если среди них имеется хотя бы один отрицательный элемент, то решение надо улучшать по правилам этапа 3.
    
&#8195;&#8195; Если же все указанные элементы неотрицательны, то в случае, когда анализируется опорное решение по целевой строке z, это означает, что на этом решении функция z достигает максимума и нужно переходить к выписыванию соответствующего оптимального плана (этап 4).
    
&#8195;&#8195; В случае поиска опорного решения (анализ по фиктивной целевой строке $\varphi$) вся строка $\varphi$ должна состоять из нулей, иначе система ограничений противоречива, и ОЗЛП не имеет решения. Получение же нулевой строки $\varphi$ свидетельствует о том, что опорное решение построено. В этом случае фиктивная целевая строка удаляется из таблицы, а решение анализируется по целевой строке z.

&#8195;&#8195; Затем нам необходимо улучшать решение по целевой строке симплекс таблицы.

In [6]:
temp1 = model1.optimize()

Начальная симплекс-таблица:


,New_X,R_S,x_1,x_2,s_1,s_2,s_3
F,nan,0,2,5,0,0,0
1,2,6,1,1,1,0,0
2,3,3,0,1,0,1,0
3,4,9,1,2,0,0,1


_________________________

Итерация : 1


,New_X,R_S,x_1,x_2,s_1,s_2,s_3
F,nan,0,2,5,0,0,0
1,2,6,1,1,1,0,0
2,3,3,0,1,0,1,0
3,4,9,1,2,0,0,1


Ведущий столбец: 3
Ведущая строка: 2
Разрешающий элемент:  1.0
_________________________

Итерация : 2


,New_X,R_S,x_1,x_2,s_1,s_2,s_3
F,nan,-2e+01,2,0,0,-5,0
1,2,3,1,0,1,-1,0
2,1,3,0,1,0,1,0
3,4,3,1,0,0,-2,1


Ведущий столбец: 2
Ведущая строка: 1
Разрешающий элемент:  1.0
_________________________

Итерация : 3


,New_X,R_S,x_1,x_2,s_1,s_2,s_3
F,nan,-2e+01,0,0,-2,-3,0
1,0,3,1,0,1,-1,0
2,1,3,0,1,0,1,0
3,4,0,0,0,-1,-1,1


_________________________

Окончательный вариант получен за  3  итераций(ии)


,New_X,R_S,x_1,x_2,s_1,s_2,s_3
F,nan,-2e+01,0,0,-2,-3,0
1,0,3,1,0,1,-1,0
2,1,3,0,1,0,1,0
3,4,0,0,0,-1,-1,1


#### Шаг №3 Проверка решения на оптимальность.

&#8195;&#8195; Проверка критерия оптимальности.

&#8195;&#8195; Среди значений индексной строки нет отрицательных. Поэтому эта таблица определяет оптимальный план задачи.

&#8195;&#8195;Окончательный вариант симплекс-таблицы:

In [7]:
model1.printTableau(temp1)

,New_X,R_S,x_1,x_2,s_1,s_2,s_3
F,nan,-2e+01,0,0,-2,-3,0
1,0,3,1,0,1,-1,0
2,1,3,0,1,0,1,0
3,4,0,0,0,-1,-1,1


#### Шаг №4 Оптимальное значение функции

&#8195;&#8195; Теперь нам осталось определить значение функции. Оптимальный план можно записать так:

In [8]:
model1.printSoln()

Коеффициенты: [3. 3.]

Оптимальное значение функции: 21.0
